<a href="https://colab.research.google.com/github/cheliu-computation/Data-Mining-CSE572/blob/main/%E2%80%9CLenet_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('/content/drive/MyDrive/AFdetect')
os.listdir()

['first_lead.csv', 'second_lead.csv', 'ECGDataDenoised', 'mnist_data']

In [ ]:
output_channels = 1
t = torch.rand(1, 1,30)

conv1 = torch.nn.Conv1d(1, 28, kernel_size=5, padding=1)
conv_t = conv1(t).view(1,1,28,28)
conv2 =torch.nn.Conv2d(1, 3, kernel_size=5)
print(conv2(conv_t).shape)
pool = torch.nn.MaxPool2d()

torch.Size([1, 3, 24, 24])


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os


In [ ]:
///////////

xla:1


In [ ]:
random_seed = 42
learning_rate = 1e-3
batch_size = 2**12
epochs = 20

img_size = 64
n_classes = 10

In [ ]:
transforms = transforms.Compose([transforms.Resize((img_size, img_size)),
                                 transforms.ToTensor()])

In [ ]:
# define transforms
# transforms.ToTensor() automatically scales the images to [0,1] range


# download and create datasets
train_dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transforms,
                               download=True)

valid_dataset = datasets.MNIST(root='mnist_data', 
                               train=False, 
                               transform=transforms)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size,
                          pin_memory = True, 
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=batch_size,
                          pin_memory = True, 
                          shuffle=False)

In [ ]:
print(train_loader.dataset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: mnist_data
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=None)
               ToTensor()
           )


# 新段落

In [ ]:
def train(train_loader, model, criterion, optimizer, device):
  '''
  training step
  '''
  # define train pattern, the parameter would change
  model.train()
  total_loss = 0
  for X, y_truth in train_loader:
    optimizer.zero_grad()
    X = X.to(device)
    y_truth = y_truth.to(device)
    
    # forward 
    y_pred,_ = model(X)
    loss = criterion(y_pred, y_truth)

    total_loss += loss.item()*X.size(0)

    # backward
    loss.backward()
    optimizer.step()
  epoch_loss = total_loss / len(train_loader.dataset)
  return model, optimizer, epoch_loss


In [ ]:
def validate(valid_loader, model, criterion, device):
  '''
  validation on each training epoch
  '''
  model.eval() # evaluation mode, parameter wont change
  total_loss = 0

  for X, y_truth in valid_loader:
    X = X.to(device)
    y_truth = y_truth.to(device)

    # forward 
    y_pred, _ = model(X)
    loss = criterion(y_pred, y_truth)
    total_loss += loss.item()*X.size(0)
  
  epoch_loss = total_loss / len(valid_loader.dataset)
  return model, epoch_loss

In [ ]:
def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

def plot_losses(train_losses, valid_losses):
    '''
    Function for plotting training and validation losses
    '''
    
    # temporarily change the style of the plots to seaborn 
    plt.style.use('seaborn')

    train_losses = np.array(train_losses) 
    valid_losses = np.array(valid_losses)

    fig, ax = plt.subplots(figsize = (8, 4.5))

    ax.plot(train_losses, color='blue', label='Training loss') 
    ax.plot(valid_losses, color='red', label='Validation loss')
    ax.set(title="Loss over epochs", 
            xlabel='Epoch',
            ylabel='Loss') 
    ax.legend()
    fig.show()
    
    # change the plot style to default
    plt.style.use('default')

In [ ]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epoch, device, print_every=2):
  '''
  training loop
  '''

  best_loss = 1e10
  train_losses, valid_losses = [],[]

  # model training
  for epoch in tqdm(range(0,epochs)):

    # training
    model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
    train_losses.append(train_loss)

    # validation
    with torch.no_grad(): # execute when gradient zero
      model, valid_loss = validate(valid_loader, model, criterion, device)
      valid_losses.append(valid_loss)
    
    if epoch%print_every == 0:
      train_acc = get_accuracy(model, train_loader, device= device)
      valid_acc = get_accuracy(model, valid_loader, device= device)

      print(f'Epoch:{epoch}\t'
         f'Trian loss{train_loss:4f}\t'
         f'Valid loss{valid_loss:4f}\t'
         f'Train acc{100*train_acc:2f}\t'
         f'Valid acc{100*valid_acc:2f}\t')
      t = torch.cuda.get_device_properties(0).total_memory
      r = torch.cuda.memory_reserved(0)/1024**3
      a = torch.cuda.memory_allocated(0)/1024**3
      print(r-a, t)
  plot_losses(train_losses, valid_losses)
  return model, optimizer, (train_losses, valid_losses)


In [ ]:
class lenet5(nn.Module):
  def __init__(self, n_classes):
    super(lenet5, self).__init__()

    self.feature_extract = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=2),
    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=2),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=2),
    nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
    nn.Tanh(),
    nn.AvgPool2d(kernel_size=11)
    )

    self.classifer = nn.Sequential(
        nn.Linear(in_features = 120, out_features =84),
        nn.Tanh(),
        nn.Linear(in_features=84, out_features=n_classes),
    )
  
  def forward(self, x):
    x = self.feature_extract(x)
    x = torch.flatten(x,1)
    logits = self.classifer(x)
    probs = F.softmax(logits, dim=1)
    
    return logits, probs

In [ ]:
torch.manual_seed(random_seed)

model = lenet5(n_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()



# 新段落

In [ ]:
model, optimizer, _ = training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device)

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
model.summary()

AttributeError: ignored